In [2]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#Initial Load boston_table_id = "opendatadbt.311.boston311"

boston311 = requests.get("https://data.boston.gov/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%22f53ebccd-bc61-49f9-83db-625f209c95f5%22%20WHERE%20%22open_dt%22%20%3E=%20%272021-03-22%27" ) 
boston311json = boston311.json() 
boston_df = pd.json_normalize(boston311json['result']['records']) 
boston_df



In [41]:
#Incremental load
boston_table_id = "opendatadbt.311.boston311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")

query_string = """
SELECT 
max(open_dt), max(closed_dt)
FROM `opendatadbt.311.boston311`
limit 10
"""

bostonmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
bostonmaxopendatedate = str(bostonmaxopendatedataframe['f0_'][0])

bostonmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
bostonmaxclosedate = str(bostonmaxclosedatedataframe['f1_'][0])


In [42]:
boston311 = requests.get("https://data.boston.gov/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%22f53ebccd-bc61-49f9-83db-625f209c95f5%22%20WHERE%20%22open_dt%22>%20%27" + bostonmaxopendatedate +"%27 OR closed_dt between '" +bostonmaxclosedate+ "' AND '" + current_datetime + "'") 
boston311json = boston311.json()
cols = ['case_enquiry_id','open_dt','closed_dt','department','subject','case_title','location_zipcode','location','neighborhood','case_status','latitude','longitude']
boston_df = pd.json_normalize(boston311json['result']['records'])
boston_df = pd.DataFrame(boston_df, columns = cols)
boston_df

,case_enquiry_id,open_dt,closed_dt,department,subject,case_title,location_zipcode,location,neighborhood,case_status,latitude,longitude
0,101003588248,2021-01-13 10:44:00,2021-04-05 11:29:32,PROP,Property Management,Graffiti: 03 0308 Other,02111,39-41 Oak St Boston MA 02111,Boston,Closed,42.3482,-71.0628
1,101003694947,2021-03-31 13:17:00,None,PWDx,Public Works Department,Request for Recycling Cart,02128,135 Chelsea St East Boston MA 02128,East Boston,Open,42.3731,-71.0348
2,101003695285,2021-03-31 16:50:00,None,BTDT,Mayor's 24 Hour Hotline,BTDT: Complaint,None,,,Open,42.3594,-71.0587
3,101003686650,2021-03-24 06:51:00,2021-03-24 12:43:37,ISD,Inspectional Services,Rodent Activity,02124,31 Wales St Dorchester MA 02124,Greater Mattapan,Closed,42.2969,-71.085
4,101003697625,2021-04-02 16:19:00,None,BWSC,Boston Water & Sewer Commission,Fire Hydrant,None,INTERSECTION of Washington St & Dale St Roxbu...,Roxbury,Open,42.3594,-71.0587
...,...,...,...,...,...,...,...,...,...,...,...,...
10344,101003700272,2021-04-05 21:05:30,None,ISD,Inspectional Services,Rodent Activity,02118,645-645A Tremont St Roxbury MA 02118,South End,Open,42.3422,-71.0753
10345,101003700286,2021-04-05 21:37:29,2021-04-05 22:13:23,BTDT,Transportation - Traffic Division,Parking Enforcement,02127,585 E Fourth St South Boston MA 02127,South Boston / South Boston Waterfront,Closed,42.3345,-71.0422
10346,101003700302,2021-04-05 22:35:22,None,BTDT,Transportation - Traffic Division,Parking Enforcement,02128,246 Bremen St East Boston MA 02128,East Boston,Open,42.3747,-71.0323
10347,101003700313,2021-04-05 22:55:10,None,PWDx,Public Works Department,Schedule Bulk Item Pickup,02122,87 Norton St Dorchester MA 02122,Dorchester,Open,42.3049,-71.0673


In [43]:

boston_table_id = "opendatadbt.311.boston311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('boston311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    boston_df, table_ref, job_config=job_config
).result()  # Make an API request.


In [5]:
boston_table_id = "opendatadbt.311.boston311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
count(*), current_date(), "Boston"
FROM `opendatadbt.311.boston311`

"""

bostondatacount = (
    client.query(query_string)
    .result()
    .to_dataframe()
)

bostondatacount = bostondatacount.rename(columns={'f0_':'count','f1_':'date','f2_':'city'})

boston_table_id = "opendatadbt.311.extractrowcount"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('extractrowcount')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

client.load_table_from_dataframe(
    bostondatacount, table_ref, job_config=job_config
).result()  # Make an API request.